In [ ]:
import torch
from torchvision import transforms
import sys
sys.path.append("data/")
sys.path.append("model/")
from data import dataloader
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from utils import plot
from tqdm import tqdm
from model import vgg16 
%load_ext autoreload
%autoreload 2

In [ ]:
train_root_dir = "archive/seg_train/seg_train"
test_root_dir = "archive/seg_test/seg_test"

train_dataset = dataloader.CustomDataset(root_dir=train_root_dir).samples
test_dataset = dataloader.CustomDataset(root_dir=test_root_dir).samples
plot(train_dataset)

In [ ]:
batch_size = 64
shuffle = True
transform = transforms.Compose([
    transforms.Resize((228, 228)),  # Resize images to 228x228
    # transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])
train_dataset = dataloader.CustomDataset(root_dir=train_root_dir, transform=transform).samples
test_dataset = dataloader.CustomDataset(root_dir=train_root_dir, transform=transform).samples

In [ ]:
pin_memory = True if torch.cuda.is_available() else False
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=pin_memory)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = vgg16.vgg16(3)
model = model.to(device)

In [ ]:

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 3
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    # Iterate over the first 100 data points
    for i, (img, label) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch + 1}')):
        print(i)
        if i >= 2:
            break  # Stop iterating after the 100th data point
        
        img, label = img.to(device), label.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(img)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        correct_predictions += (predicted == label).sum().item()
        total_samples += label.size(0)
        running_loss += loss.item()
        
        if (i + 1) % 100 == 0:  # print every 100 mini-batches 
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0
    
    # Calculate accuracy after processing all batches for the epoch
    accuracy = correct_predictions / total_samples
    print(f'Accuracy after epoch {epoch + 1}: {accuracy * 100:.2f}%')

print('Finished Training')